In [1]:
!pip install qiskit matplotlib numpy -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.1 MB/s eta 0:00:00



import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile

# Parameters
N_meas = 20
p_values = np.linspace(0, 1, 15)
survival = []

backend = AerSimulator()

for p in p_values:
    qc = QuantumCircuit(1, 1)
    qc.h(0)  # superposition
    
    for _ in range(N_meas):
        if np.random.rand() < p:
            qc.measure(0, 0)
            qc.reset(0)
            qc.h(0)
    
    qc.measure(0, 0)
    
    # NEW: use transpile + run
    qc_compiled = transpile(qc, backend)
    job = backend.run(qc_compiled, shots=10000)
    result = job.result()
    counts = result.get_counts()
    prob_stay = counts.get('0', 0) / 10000
    survival.append(prob_stay)

# Theory
theory = np.sqrt(1 - p_values)

# Plot
plt.figure(figsize=(8,5))
plt.plot(p_values, survival, 'o-', label='My Sim (Shaurya Singh)')
plt.plot(p_values, theory, '--', label='Theory √(1-p)')
plt.xlabel('Measurement Strength p')
plt.ylabel('Survival Probability')
plt.title('Quantum Zeno Effect - 15yo India Build')
plt.legend()
plt.grid()
plt.savefig('zeno_plot.png')
plt.show()
